In [1]:
from extremeweatherbench import metrics
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os


In [8]:
# lets start by defining a function to generate a T x N x M array since this will be commonly used
T = 24
L = 6
N = 400
M = 600
# set seed for reproducibility
np.random.seed(42)
def _generate_input(dims, mu, sigma):
    return np.random.normal(loc=mu, scale=sigma, size = dims)

# let's generate obs using the standard normal distribution N(0,1) for a T x N x M field
obs = _generate_input((T, N, M), 0.0, 1.0)

# now for the fcst we'll need to generate and stack the leadtime arrays
# since we only have 6 lead times we can be a bit lazy and use a for loop
# we will also increase the bias and variance as we increase the leadtime
var_inc = 0.5
bias_inc = 0.25
var, bias = 1.0, 0.0
fcst = []
for i in range(L):
    var += var_inc
    bias += bias_inc
    fcst.append(
        _generate_input((T, N, M), bias, var)
    )
fcst = np.stack(fcst)

# we now have the arrays in raw numpy format, we'll need some annotations
# lets convert it to xarray and check that our final dimensions match
da_obs = xr.DataArray(
    data=obs,
    dims=["time", "latitude", "longitude"],
    coords=dict(
        time=range(T),
        latitude=range(N),
        longitude=range(M),
    ),
    attrs=dict(
        description="observations",
    ),
)

da_fcst = xr.DataArray(
    data=fcst,
    dims=["lead_time", "time", "latitude", "longitude"],
    coords=dict(
        lead_time=range(L),
        time=range(T),
        latitude=range(N),
        longitude=range(M),
    ),
    attrs=dict(
        description="forecast",
    ),
)

In [9]:
fss = metrics.FSS()
fss.compute(da_obs, da_fcst)

<xarray.DataArray (lead_time: 6)> Size: 48B
array([0.80066187, 0.77794228, 0.76082317, 0.74868834, 0.73973474,
       0.73277501])
Coordinates:
  * lead_time  (lead_time) int64 48B 0 1 2 3 4 5